In [36]:
from binance.um_futures import UMFutures
import plotly.graph_objects as go
import requests
import json
import numpy as np
import pandas as pd
import pandas_ta as ta
import strats
import vectorbt as vbt
from pandas import Timestamp
from sklearn.linear_model import LinearRegression
import datetime
from chart import Chart
from data_ops import DataGetter
data_getter=DataGetter("iPQMe46exUy10KiBaBahQT7ow1uzb9jaxlKj19Bg5BI8JEwJL5bw9LCvJtfKuVbP","J4FGJQdGPpNGm8jpnj3IuFuDJuAe7FIa3cM51L4z662UYux3qD0ByLJ0bfIltZvK")

In [62]:
whitelist = open("whitelist.txt","r")
asset_dict_15m = dict()

for line in whitelist:
    asset_dict_15m.update({line[:-1] : data_getter.get_pair_data(line[:-1],"15m")})

In [37]:
data = data_getter.get_pair_data(pair_in="CHZUSDT",interval_in="1m")

In [38]:
chz_chart = Chart(data)

In [39]:
chz_chart.draw_ema(data,order= 100)

In [40]:
chz_chart.draw_supertrend(data,14)

In [41]:
ta.momentum.eri(data["High"],data["Low"],data["Close"])

,BULLP_13,BEARP_13
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
495,0.000308,0.000078
496,0.000240,0.000040
497,0.000321,0.000051
498,0.000124,-0.000406


In [42]:
chz_chart.plot()

In [43]:
#Long_PNL.value().vbt.plot() portfolio plotting function
#Long_PNL.plot().show()

In [44]:
#SIGNALS
bollinger_signals = strats.bollinger_strat(data)
donchian_signals = strats.donchian_strat(data)
rsi_signals = strats.rsi_strat(data)
supertrend_signals = strats.supertrend_strat(data)

In [45]:
bollinger_backtest = strats.backtest(data["Close"],bollinger_signals)
donchian_backtest = strats.backtest(data["Close"], donchian_signals)
rsi_backtest = strats.backtest(data["Close"],rsi_signals)
supertrend_backtest = strats.backtest(data["Close"],supertrend_signals)

In [46]:
donchian_backtest.plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '4d0f9c63-433c-466e-b037-1dd156be3deb',
              'x': array([datetime.datetime(2024, 3, 15, 21, 30),
                          datetime.datetime(2024, 3, 15, 21, 31),
                          datetime.datetime(2024, 3, 15, 21, 32), ...,
                          datetime.datetime(2024, 3, 16, 5, 47),
                          datetime.datetime(2024, 3, 16, 5, 48),
                          datetime.datetime(2024, 3, 16, 5, 49)], dtype=object),
              'xaxis': 'x',
              'y': array([0.14388, 0.14412, 0.14407, ..., 0.14392, 0.1435 , 0.14373]),
              'yaxis': 'y'},
             {'customdata': array([[  0.        , 691.70643979,   0.        ]]),
              'hovertemplate': ('Order Id: %{customdata[0]}<br>' ... '<br>Fees: %{customdata[2]:.6f}'),
     

In [47]:
bollinger_backtest.plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '7f9fd582-2c0b-4e40-b635-3002e5cee492',
              'x': array([datetime.datetime(2024, 3, 15, 21, 30),
                          datetime.datetime(2024, 3, 15, 21, 31),
                          datetime.datetime(2024, 3, 15, 21, 32), ...,
                          datetime.datetime(2024, 3, 16, 5, 47),
                          datetime.datetime(2024, 3, 16, 5, 48),
                          datetime.datetime(2024, 3, 16, 5, 49)], dtype=object),
              'xaxis': 'x',
              'y': array([0.14388, 0.14412, 0.14407, ..., 0.14392, 0.1435 , 0.14373]),
              'yaxis': 'y'},
             {'customdata': array([[   0.        ,  694.25159678,    0.        ],
                                   [   2.        , 1385.43192085,    0.        ],
                          

In [48]:
rsi_backtest.plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '9045d949-a3e8-4672-9b97-dfe051da6e11',
              'x': array([  0,   1,   2, ..., 497, 498, 499], dtype=int64),
              'xaxis': 'x',
              'y': array([0.14388, 0.14412, 0.14407, ..., 0.14392, 0.1435 , 0.14373]),
              'yaxis': 'y'},
             {'customdata': array([[  0.       , 695.0236308,   0.       ]]),
              'hovertemplate': ('Order Id: %{customdata[0]}<br>' ... '<br>Fees: %{customdata[2]:.6f}'),
              'legendgroup': '1',
              'marker': {'color': '#37B13F',
                         'line': {'color': 'rgb(38,123,44)', 'width': 1},
                         'size': 8,
                         'symbol': 'triangle-up'},
              'mode': 'markers',
              'name': 'Buy',
              'type': 'scatter',
           

In [49]:
supertrend_backtest.plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'e9fd1738-2068-4807-8a38-301a8baffb7e',
              'x': array([  0,   1,   2, ..., 497, 498, 499], dtype=int64),
              'xaxis': 'x',
              'y': array([0.14388, 0.14412, 0.14407, ..., 0.14392, 0.1435 , 0.14373]),
              'yaxis': 'y'},
             {'customdata': array([[   0.        ,  693.86622259,    0.        ],
                                   [   2.        , 1381.41717652,    0.        ],
                                   [   4.        , 1367.63109769,    0.        ],
                                   [   6.        , 1359.33047172,    0.        ],
                                   [   8.        , 1350.67784205,    0.        ],
                                   [  10.        , 1337.7222808 ,    0.        ],
                                   [

In [50]:
supertrend_backtest.max_drawdown()

-0.0556990133136942

In [51]:
"""
data_getter.client.new_order_test(
    symbol="BTCUSDT",
    side="BUY",  # Valid options: "BUY" or "SELL"
    type="LIMIT",  # Order type (optional, default might be "LIMIT")
    quantity=0.01,  # Quantity of the asset to buy/sell
    price=70000,  # Price for LIMIT orders (optional)
    timeInForce="GTC",  # Order validity (optional, "GTC" - Good Till Cancelled)
    recvWindow=50000  # Allowed time window for request validity (in milliseconds)
)
"""

'\ndata_getter.client.new_order_test(\n    symbol="BTCUSDT",\n    side="BUY",  # Valid options: "BUY" or "SELL"\n    type="LIMIT",  # Order type (optional, default might be "LIMIT")\n    quantity=0.01,  # Quantity of the asset to buy/sell\n    price=70000,  # Price for LIMIT orders (optional)\n    timeInForce="GTC",  # Order validity (optional, "GTC" - Good Till Cancelled)\n    recvWindow=50000  # Allowed time window for request validity (in milliseconds)\n)\n'

In [52]:
st = strats.supertrend_strat(data)
st

,Long Signal,Short Signal,Exit Signal
0,False,False,True
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
495,True,False,False
496,True,False,False
497,True,False,False
498,True,False,False
